In [1]:
#Standard Imports
import pandas as pd
import numpy as np
import os
import seaborn as sns

In [70]:
############################################################
# Set up and read in file                                  # 
# path is filepath                                         #
# cols is the columns from the file that should be read in # 
# stats stores all of the advance stats                    #
############################################################

path = 'C:\\Users\\dhoth\\Documents\\Quizbowl\\Advance Stats\\'
cols = ['Team','Player','Category','Subcategory','Round','Answer','Buzzpoint','SUM of Buzz value','SUM of Buzz %']
stats = pd.read_excel(path+"\\Fall Open\\2019_Fall_Open.xlsx", usecols = cols, index_col = [0,1,2,3]).reset_index()

In [71]:
###########################################################
# Loads in how many games players or teams played         #
# Uncomment first two if you want to read it in from a    #
# spreadsheet.                                            #
###########################################################

t = pd.DataFrame(stats.groupby(['Team']).nunique()['Round'])
t.loc['4th Place']
df_games = pd.DataFrame(stats.groupby(['Team','Player']).nunique()['Round']).reset_index()
df_games['Round'] = df_games.apply(lambda row: t.loc[row.Team], axis = 1)

df_games = df_games.rename(columns = {'Round': 'GP'})
df_games['TUH'] = df_games['GP'] * 20

# df_games = pd.read_excel(path+'\\Fall Open\\2019_Fall_Open_Players.xlsx', index_col = 0)
# df_games['GP'] = df_games['GP'].astype(int)
# df_games.head()

In [8]:
###########################################################
# Removes negs from the dataframe. Also removes buzzes    #
# with bad buzzpoint data.                                #
###########################################################

stats = stats[(stats['SUM of Buzz value'] == 10) | (stats['SUM of Buzz value'] == 15)]
for name,grp in stats.groupby('Player'): #Toggle for Team vs Players
    df_games.loc[name,'TUH'] = df_games.loc[name,'TUH'] - (grp['SUM of Buzz %'] == 0).sum()

stats = stats[stats['SUM of Buzz %'] > 0] 

In [10]:
###########################################################
# Computes BPA.                                           #
# buzzpoints is the dataframe that stores buzzpoints.     #
# subject takes in a specific category if you want.       #
# Team determines if you want a Team's BPA or Player BPA. #
###########################################################

def compute_BPA(buzzpoints = stats, subject = None, team = False):
    buzzpoints = stats[(stats['SUM of Buzz value'] == 10) | (stats['SUM of Buzz value'] == 15)]
    buzzpoints = buzzpoints[buzzpoints['SUM of Buzz %'] > 0]
    if subject != None:
        buzzpoints = buzzpoints[buzzpoints.Category == subject]
    if team is False:
        df_grouped = buzzpoints.groupby(['Player'])
    else: 
        df_grouped = buzzpoints.groupby(['Team'])
    df_processed = df_grouped.mean()
    if team is False:
        df_processed['Buzzes'] = df_grouped.count()['Team']
    if team is True:
        df_processed['Buzzes'] = df_grouped.count()['Player']
    if team is False:
        df_processed = df_processed.join(df_games, on = 'Player')
    else:
        df_processed = df_processed.join(df_games, on = 'Team')
    df_processed['BPA'] = ((1-df_processed['SUM of Buzz %'])*df_processed['Buzzes'])/df_processed['TUH']
    if subject != None:
        df_processed['BPA'] = sub_mult[subject]*df_processed['BPA']
    return df_processed[['Buzzpoint','SUM of Buzz %','BPA']]

In [11]:
###########################################################
# Computes overall and subject BPAs. Set team = False for #
# individual BPA. Results are outputed to Excel files.    #
# sub_mult is used to compute subject BPAs. The entry for #
# each category is 20/(# of questions per packet)         #
###########################################################

sub_mult = {'History': 5, 'Literature':5,'Science':5,'Arts':6.666,'Other':15,
            'RMPSS':5.454545}

cats = [None, 'Science','Literature','History','Other', 'RMPSS']
for c in cats:
    d = compute_BPA(team = True, subject = c)
    d = d.sort_values(by = 'BPA',ascending = False)

    try: 
        os.chdir(path+'Fall Open\\')
    except OSError: 
        print("Can't change the Current Working Directory")
    if c is not None:    
        d.to_excel('2019_Fall_Open_Team_{}_BPA.xlsx'.format(c))
    else:
        d.to_excel('2019_Fall_Open_Team_BPA.xlsx')